# One-Dimensional Vertical Flow

This notebook sets up a simple one-dimensional vertical flow problem.  The purpose is to understand how vertical flow is affected by head and density.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import flopy
import numpy as np

In [ ]:
def build_models(ws, name, nlay, hghb_upper, hghb_lower, conc_init):
    lx = 1.0
    lz = 100.0
    nrow = 1
    ncol = 1
    nper = 1
    delc = 1.0
    delr = lx / ncol
    delz = lz / nlay
    top = -10.0
    botm = [top - k * delz for k in range(1, nlay + 1)]

    perlen = [1.0]
    nstp = [1]
    tsmult = [1.0]

    Kh = 20.0
    Kv = 20.0

    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

    nouter, ninner = 20, 10
    hclose, rclose, relax = 1e-8, 1e-6, 0.97

    # build MODFLOW 6 files
    sim = flopy.mf6.MFSimulation(
        sim_name=name, version="mf6", exe_name="mf6", sim_ws=ws
    )
    # create tdis package
    tdis = flopy.mf6.ModflowTdis(sim, time_units="DAYS", nper=nper, perioddata=tdis_rc)

    # create gwf model
    gwfname = "gwf_" + name
    gwtname = "gwt_" + name

    gwf = flopy.mf6.ModflowGwf(
        sim,
        save_flows=True,
        modelname=gwfname,
        model_nam_file=f"{gwfname}.nam",
    )

    imsgwf = flopy.mf6.ModflowIms(
        sim,
        print_option="ALL",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename=f"{gwfname}.ims",
    )

    idomain = np.full((nlay, nrow, ncol), 1)
    dis = flopy.mf6.ModflowGwfdis(
        gwf,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        idomain=idomain,
    )

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(gwf, strt=0.0)

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        xt3doptions=False,
        save_flows=True,
        save_specific_discharge=True,
        icelltype=0,
        k=Kh,
        k33=Kv,
    )

    pd = [(0, 0.7, 0.0, gwtname, "CONCENTRATION")]
    density_filerecord = f"{gwfname}.buy.dens"
    buy = flopy.mf6.ModflowGwfbuy(
        gwf, denseref=1000.0, packagedata=pd, density_filerecord=density_filerecord
    )

    # put ghb in top layer
    ghbconc = conc_init[0]
    ghb1 = [
        [(0, 0, 0), hghb_upper, 10.0, ghbconc, top],
    ]
    ghb_upper = flopy.mf6.ModflowGwfghb(
        gwf,
        stress_period_data=ghb1,
        print_input=True,
        print_flows=True,
        pname="GHB-UPPER",
        auxiliary=["CONCENTRATION", "ELEVATION"],
        filename=f"{gwfname}.upper.ghb",
    )

    # put ghb in bottom layer
    ghbconc = conc_init[0]
    ghb2 = [
        [(nlay - 1, 0, 0), hghb_lower, 10.0, ghbconc, botm[-1]],
    ]
    ghb_lower = flopy.mf6.ModflowGwfghb(
        gwf,
        stress_period_data=ghb2,
        print_input=True,
        print_flows=True,
        pname="GHB-LOWER",
        auxiliary=["CONCENTRATION", "ELEVATION"],
        filename=f"{gwfname}.lower.ghb",
    )

    # output control
    oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=f"{gwfname}.bud",
        head_filerecord=f"{gwfname}.hds",
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )

    # create gwt model
    gwt = flopy.mf6.ModflowGwt(
        sim,
        modelname=gwtname,
        model_nam_file=f"{gwtname}.nam",
    )

    imsgwt = flopy.mf6.ModflowIms(
        sim,
        print_option="ALL",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename=f"{gwtname}.ims",
    )
    sim.register_ims_package(imsgwt, [gwt.name])

    dis = flopy.mf6.ModflowGwtdis(
        gwt,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        idomain=idomain,
    )

    # initial conditions
    ic = flopy.mf6.ModflowGwtic(
        gwt,
        strt=conc_init,
        filename=f"{gwtname}.ic",
    )

    # advection
    adv = flopy.mf6.ModflowGwtadv(gwt, scheme="UPSTREAM", filename=f"{gwtname}.adv")

    # storage
    porosity = 0.2
    mst = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity, filename=f"{gwtname}.sto")
    # sources
    sourcerecarray = [
        # ("CHD-1", "AUX", "CONCENTRATION"),
        ("GHB-UPPER", "AUX", "CONCENTRATION"),
        ("GHB-LOWER", "AUX", "CONCENTRATION"),
    ]
    ssm = flopy.mf6.ModflowGwtssm(
        gwt, sources=sourcerecarray, filename=f"{gwtname}.ssm"
    )

    # output control
    oc = flopy.mf6.ModflowGwtoc(
        gwt,
        budget_filerecord=f"{gwtname}.cbc",
        concentration_filerecord=f"{gwtname}.ucn",
        concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
        printrecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
    )

    # GWF GWT exchange
    gwfgwt = flopy.mf6.ModflowGwfgwt(
        sim,
        exgtype="GWF6-GWT6",
        exgmnamea=gwfname,
        exgmnameb=gwtname,
        filename=f"{name}.gwfgwt",
    )

    return sim

## Generate and Run Model

In [ ]:
name = "vflow"
ws = "./temp/vflow"
nlay = 10
hghb_upper = 10.0
hghb_lower = 10.0
conc_init = [35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]

sim = build_models(ws, name, nlay, hghb_upper, hghb_lower, conc_init)
sim.write_simulation()
success, buff = sim.run_simulation(silent=True, report=True)
assert success, "MODFLOW 6 did not terminate normally."

## Process Output

In [ ]:
# extract data
gwf = sim.gwf[0]
gwt = sim.gwt[0]
head = gwf.output.head().get_data()
conc = gwt.output.concentration().get_data()
density = gwf.buy.output.density().get_data().flatten()

spdis = gwf.output.budget().get_data(text="DATA-SPDIS")[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

# Get flows from each GHB package separately using paknam parameter
ghbflow_upper = gwf.output.budget().get_data(text="GHB", paknam2="GHB-UPPER")[-1]
ghbflow_lower = gwf.output.budget().get_data(text="GHB", paknam2="GHB-LOWER")[-1]

# print ghb flows (positive flow is into the model)
print("GHB-UPPER flow: ", ghbflow_upper["q"])
print("GHB-LOWER flow: ", ghbflow_lower["q"])

# print a table of layer results
pd.DataFrame(
    {
        "layer": list(range(nlay)),
        "head": head.flatten(),
        "cstrt": gwt.ic.strt.array.flatten(),
        "concentration": conc.flatten(),
        "density": density,
        "qz": qz.flatten(),
    }
)

In [ ]:
# f = plt.figure(figsize=(2,8))
ax = plt.subplot(1, 1, 1)
pxs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 0}, ax=ax)
img = pxs.plot_array(gwt.ic.strt.array, vmin=0, vmax=conc.max(), cmap="jet")
pxs.plot_grid()
pxs.plot_vector(qx, qy, qz)
ax.set_xlabel("x")
ax.set_ylabel("z")
plt.colorbar(img, shrink=0.5);